# Galerkin-based Solution of Random Differential Equation

This tutorial demonstrates how random differential equations can be solved using polynomial chaos expansions (PCE).

## Theory

A random differential equation is an ordinary differential equation that has random parameters, hence its solution is itself a (time-varying) random variable.
Perhaps the simplest non-trivial example is the following scalar, linear ordinary differential equation
$$
\dot{x}(t) = a x(t), \quad x(0) = x_{0},
$$
where $a$ is the realization of a Gaussian random variable $\mathsf{a} \sim \mathcal{N}(\mu, \sigma^2)$ with mean $\mu$ and variance $\sigma^2$.
Arguably, for every realization $a$ we can solve the differential equation and obtain
$$
x(t) = x_0 \mathrm{e}^{a t},
$$
from which we find that
$$
\ln (x(t)) = \ln (x_0) + at \sim \mathcal{N}(\ln(x_0) + \mu t, (\sigma t)^2).
$$
In other words, the logarithm of the solution is normally distributed (so-called [log-normal distribution](https://en.wikipedia.org/wiki/Log-normal_distribution)).

We'd like to obtain this result numerically with the help of PCE.
The first step is to define the (truncated) PCE for the random variable $\mathsf{a}$
$$
\mathsf{a} = \sum_{i=0}^{L} a_i \phi_i,
$$
where $a_i$ are the so-called PCE coefficients, and $\phi_i$ are the orthogonal basis polynomials.
As the solution to the random differential equation is itself a random variable, we treat $x(t)$ as the realization of the random variable $\mathsf{x}(t)$, and define its PCE
$$
\mathsf{x}(t) = \sum_{i=0}^{L} x_i(t) \phi_i.
$$
The question is how to obtain the unknown PCE coefficients $x_i(t)$ from the known PCE coefficients $a_i$ relative to the orthogonal basis polynomials $\phi_i$.
This can be done using Galerkin projection, which is nothing else than projecting onto the orthogonal basis.
Think of a three-dimensional space, in which you have placed some three-dimensional object.
If you know project the silhouett of the object onto every axis of the three-dimensional space, then you are doing a Galerkin projection.
With PCE the concept is equivalent, but the imagination has a harder time.
The first step for Galerkin projection is to insert the PCEs
$$
\sum_{i=0}^{L} \dot{x}_i(t) \phi_i = \sum_{j=0}^{L} a_j \phi_j \sum_{k=0}^{L} x_k(t) \phi_k;
$$
the second step is to project onto every basis polynomial $\phi_m$ for $m = 0, 1, \dots, L$, and to exploit orthogonality of the basis.
This gives
$$
\dot{x}_m(t) \langle \phi_m, \phi_m \rangle = \sum_{j=0}^{L} \sum_{k=0}^{L} a_j x_k(t) \langle \phi_l \phi_k, \phi_m \rangle \quad m = 0, 1, \dots, L.
$$
Of course, the initial condition must not be forgotten:
$$
x_0(0) = x_0, \quad x_m(0) = 0 \quad m = 1, \dots, L.
$$
If we can solve this enlarged system of ordinary random differential equations, we can reconstruct the analytic solution.

## Practice
We begin by defining the random differential equation

In [4]:
x0 = 2.0
μ, σ = -0.5, 0.05 
tend, Δt = 3.0, 0.01

(3.0, 0.01)

Next, we define an orthogonal basis (and its quadrature rule) relative to the Gaussian measure using `PolyChaos`.
We choose a maximum degree of `L`.

In [5]:
using PolyChaos
L, Nrec = 6, 40
opq = GaussOrthoPoly(L;Nrec=Nrec)

GaussOrthoPoly(6, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0  …  30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0], GaussMeasure(PolyChaos.w_gaussian, (-Inf, Inf), true), Quad("golubwelsch", 39, [-11.2897, -10.3133, -9.50131, -8.77344, -8.09915, -7.46268, -6.85455, -6.26849, -5.70006, -5.14596  …  5.14596, 5.70006, 6.26849, 6.85455, 7.46268, 8.09915, 8.77344, 9.50131, 10.3133, 11.2897], [9.44334e-29, 2.78479e-24, 7.59245e-21, 5.3707e-18, 1.48613e-15, 1.99873e-13, 1.49172e-11, 6.74865e-10, 1.96987e-8, 3.88412e-7  …  3.88412e-7, 1.96987e-8, 6.74865e-10, 1.49172e-11, 1.99873e-13, 1.48613e-15, 5.3707e-18, 7.59245e-21, 2.78479e-24, 9.44334e-29]))

Now we can define the PCE for $\mathsf{a}$ and solve the Galerkin-projected ordinary differential equation using `DifferentialEquations.jl`.

In [6]:
using DifferentialEquations

a = [ convert2affinePCE(μ,σ,opq); zeros(Float64,L-1) ] # PCE coefficients of a
xinit = [ x0; zeros(Float64,L) ] # PCE coefficients of initial condition

t2 = Tensor(2,opq); # \langle \phi_i, \phi_j \rangle
t3 = Tensor(3,opq); # \langle \phi_i \phi_j, \phi_k \rangle

# Galerkin-projected random differential equation
function ODEgalerkin(du,u,p,t)
   du[:] = [ sum( p[j+1]*u[k+1]*t3.get([j,k,m])/t2.get([m,m]) for j=0:L for k=0:L) for m=0:L ] 
end

probgalerkin = ODEProblem(ODEgalerkin,xinit,(0,tend),a)
solgalerkin = solve(probgalerkin;saveat=0:Δt:tend)
t, x = solgalerkin.t, solgalerkin.u;

For later purposes we compute the expected value and the standard deviation at all time instants using PCE.

In [7]:
# an advantage of PCE is that moments can be computed from the PCE coefficients alone; no sampling required
mean_pce = [ mean(x[i],opq) for i=1:length(x)]  
std_pce = [ std(x[i],opq) for i=1:length(x) ]

301-element Array{Float64,1}:
 0.0                  
 0.0009950126657575441
 0.0019801011629580713
 0.0029553408323050867
 0.003920806490164937 
 0.004876572450341695 
 0.005822712524165987 
 0.006759300020584717 
 0.007686407746251699 
 0.00860410800561929  
 0.00951247260103081  
 0.010411572905029297 
 0.01130147989492374  
 ⋮                    
 0.0692065255294234   
 0.06910714081071131  
 0.0690071043857249   
 0.06890642513422096  
 0.06880511185811229  
 0.06870317328148076  
 0.06860061805059041  
 0.06849745473390102  
 0.0683936918220815   
 0.06828933772802367  
 0.06818440078685599  
 0.06807888925595759  

We compare the solution from PCE to a Monte-Carlo-based solution.
That means to solve the ordinary differential equation for many samples of $\mathsf{a}$.
We first sample from the measure using `sampleMeasure`, and then generate samples of $\mathsf{a}$ using `evaluatePCE`.
After that we solve the ODE and store the results in `xmc`.

In [8]:
using Statistics
Nsmpl = 5000
ξ = sampleMeasure(Nsmpl,opq)     # sample from Gaussian measure; effectively randn() here    
asmpl = evaluatePCE(a,ξ,opq)     # sample random variable with PCE coefficients a; effectively μ + σ*randn() here
# or: asmpl = samplePCE(Nsmpl,a,opq)
xmc = [ solve(ODEProblem((u,p,t)->aa*u,x0,(0,tend));saveat=0:Δt:tend).u for aa in asmpl]
xmc = hcat(xmc...);

Now we can compare the Monte Carlo mean and standard deviation to the expression from PCE for every time instant.

In [9]:
[ mean(xmc,dims=2)-mean_pce std(xmc,dims=2)-std_pce]

301×2 Array{Float64,2}:
 0.0          0.0        
 2.13335e-6   1.81995e-5 
 4.26359e-6   3.62305e-5 
 6.39064e-6   5.4094e-5  
 8.5144e-6    7.17914e-5 
 1.06348e-5   8.93237e-5 
 1.27516e-5   0.000106692
 1.48649e-5   0.000123898
 1.69744e-5   0.000140942
 1.90801e-5   0.000157826
 2.11818e-5   0.00017455 
 2.32797e-5   0.000191117
 2.53731e-5   0.000207527
 ⋮                       
 0.000334613  0.0013965  
 0.000334785  0.00139495 
 0.000334952  0.00139338 
 0.000335112  0.00139179 
 0.000335266  0.00139019 
 0.000335415  0.00138858 
 0.000335558  0.00138695 
 0.000335696  0.00138531 
 0.000335828  0.00138365 
 0.000335955  0.00138198 
 0.000336076  0.0013803  
 0.000336192  0.00137861 

Clearly, the accuracy of PCE deteriorates over time.
Possible remedies are to increase the dimension of PCE, and to tweak the tolerances of the integrator.

Finally, we compare whether the samples follow a log-normal distribution, and compare the result to the analytic mean and standard deviation.

In [10]:
logx_pce = [ log.(evaluatePCE(x[i],ξ,opq)) for i=1:length(t)]
[mean.(logx_pce)-(log(x0) .+ μ*t) std.(logx_pce)-σ*t ]

301×2 Array{Float64,2}:
 -6.66134e-16  6.662e-16  
  1.06746e-6   9.14208e-6 
  2.1349e-6    1.82842e-5 
  3.20234e-6   2.74263e-5 
  4.26979e-6   3.65683e-5 
  5.33725e-6   4.57104e-5 
  6.40472e-6   5.48525e-5 
  7.47219e-6   6.39946e-5 
  8.53966e-6   7.31367e-5 
  9.60712e-6   8.22787e-5 
  1.06746e-5   9.14208e-5 
  1.1742e-5    0.000100563
  1.28091e-5   0.000109705
  ⋮                       
  0.000308754  0.00264189 
  0.000309829  0.00265103 
  0.000310903  0.00266017 
  0.000311976  0.00266931 
  0.000313048  0.00267845 
  0.000314118  0.00268759 
  0.000315188  0.00269673 
  0.000316257  0.00270587 
  0.000317325  0.00271501 
  0.000318393  0.00272415 
  0.00031946   0.00273329 
  0.000320527  0.00274243 